## Objetivos y Alcance del Sistema Híbrido

- Elevar el AUC-ROC a 0.870-0.880 manteniendo estabilidad y auditabilidad.
- Cubrir 15-25% de los casos con reglas determinísticas para reducir dependencia de modelos.
- Reutilizar el pipeline validado e incorporar cinco variables derivadas para enriquecer la señal.

### Arquitectura en Tres Capas
1. **Feature engineering dirigido**: DelinquencySum, SevereDelinquencyFlag, IncomePerDependent, CreditExposure, UtilizationDelinquencyMix.
2. **Modelos complementarios**: XGBoost base sin restricciones y LightGBM monotónico con coherencia causal.
3. **Gateway basado en reglas**: Clasifica casos evidentes, delega zonas grises al modelo monotónico y refuerza decisiones de alta confianza del baseline.

In [9]:
# Sección 0: Configuración y Librerías
import warnings
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    classification_report,
    confusion_matrix
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

ROOT = Path(r'c:/MachineLearningPG')
DATA_PATH = ROOT / 'data' / 'processed_for_modeling.csv'
MODELS_DIR = ROOT / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
TARGET = 'SeriousDlqin2yrs'
RANDOM_STATE = 42

print('Configuración lista')

Configuración lista


## Sección 1 — Carga de Datos y Feature Engineering

Partimos del dataset preprocesado y generamos **5 variables derivadas** antes del split train/test para garantizar que ambos conjuntos compartan la misma transformación:

### Construcción de Variables Sintéticas

1. **DelinquencySum** = Suma de `NumberOfTimes90DaysLate` + `NumberOfTime60-89DaysPastDueNotWorse` + `NumberOfTime30-59DaysPastDueNotWorse`

   - *Razón*: Captura severidad acumulada; un cliente con múltiples atrasos menores puede ser tan riesgoso como uno con un atraso grave.

2. **SevereDelinquencyFlag** = 1 si `NumberOfTimes90DaysLate ≥ 2`, 0 en caso contrario

   - *Razón*: Señal binaria de comportamiento crítico repetido; facilita reglas determinísticas posteriores.

3. **IncomePerDependent** = `MonthlyIncome` / (`NumberOfDependents` + 1)

   - *Razón*: Ajusta capacidad de pago por carga familiar; ingresos altos con muchos dependientes reducen resiliencia.
   - *Manejo de nulos*: Si `NumberOfDependents=0`, se evita división por cero sumando 1; si `MonthlyIncome` es nulo, se imputa con la mediana.

4. **CreditExposure** = `DebtRatio` × `MonthlyIncome`

   - *Razón*: Convierte ratio relativo en monto absoluto; un `DebtRatio=0.5` con ingresos de $10k es más manejable que con $2k.

5. **UtilizationDelinquencyMix** = `RevolvingUtilizationOfUnsecuredLines` × (`DelinquencySum` + 1)

   - *Razón*: Interacción entre utilización de crédito y morosidad; alta utilización + historial limpio es menos grave que alta utilización + atrasos previos.

### Resultado

Dataset enriquecido con **15 variables originales + 5 derivadas = 20 features** listas para modelado.

In [10]:
assert DATA_PATH.exists(), f'No se encontró {DATA_PATH}'
df = pd.read_csv(DATA_PATH)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

df = df.copy()
df['DelinquencySum'] = df[['NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTime30-59DaysPastDueNotWorse']].sum(axis=1)
df['SevereDelinquencyFlag'] = (df['NumberOfTimes90DaysLate'] >= 2).astype(int)
income_base = df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())
dependents = df['NumberOfDependents'].replace(0, np.nan)
df['IncomePerDependent'] = income_base / (dependents + 1)
df['IncomePerDependent'] = df['IncomePerDependent'].fillna(income_base)
df['CreditExposure'] = df['DebtRatio'] * income_base
df['UtilizationDelinquencyMix'] = df['RevolvingUtilizationOfUnsecuredLines'] * (df['DelinquencySum'] + 1)

feature_columns = [col for col in df.columns if col != TARGET]

print(f'Dataset enriquecido con {len(feature_columns)} variables (incluye 5 nuevas).')
df.head()

Dataset enriquecido con 21 variables (incluye 5 nuevas).


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,...,NumberOfDependents_na,RevolvingUtilizationOfUnsecuredLines_clipped,RevolvingUtilizationOfUnsecuredLines_log,DebtRatio_clipped,DebtRatio_log,DelinquencySum,SevereDelinquencyFlag,IncomePerDependent,CreditExposure,UtilizationDelinquencyMix
0,1,-0.021150,-0.493860,2.516600,-0.172833,0.209579,0.883657,-0.186131,4.409546,-0.196207,...,-0.163899,1.265454,1.307305,-0.348137,-0.356179,2.134261,0,0.097910,-0.036222,-0.066290
1,0,-0.020385,-0.832342,-0.351898,-0.173168,-0.296226,-0.865297,-0.186131,-0.901283,-0.196207,...,-0.163899,1.807904,1.723114,-0.348888,-0.537442,-0.734236,0,-0.239432,0.051297,-0.005418
2,0,-0.021582,-0.967735,1.082351,-0.173186,-0.261937,-1.253953,1.875277,-0.901283,-0.196207,...,-0.163899,0.958919,1.051959,-0.348928,-0.550171,2.761421,0,-0.784537,0.045364,-0.081180
3,0,-0.023281,-1.509307,-0.351898,-0.173210,-0.241922,-0.670969,-0.186131,-0.901283,-0.196207,...,-0.163899,-0.246161,-0.144904,-0.348982,-0.567849,-0.734236,0,-0.724590,0.041903,-0.006187
4,0,-0.020585,-0.223074,1.082351,-0.173215,4.435064,-0.282312,-0.186131,-0.016145,-0.196207,...,-0.163899,1.666171,1.618523,-0.348995,-0.571973,0.700012,0,13.283649,-0.768221,-0.034995


## Sección 2 — Split y Configuración de Modelos

### División Estratificada 70/30
- `train_test_split(..., stratify=y)` mantiene el ~6.7% de morosos en ambos conjuntos.
- Todas las transformaciones se entrenan con `X_train` y se aplican luego sobre `X_test`.

### Restricciones Monotónicas
| Variable | Restricción | Justificación breve |
|----------|-------------|---------------------|
| RevolvingUtilizationOfUnsecuredLines | +1 | Mayor utilización implica más riesgo |
| Age | -1 | Más edad suele correlacionar con estabilidad |
| DebtRatio | +1 | Carga de deuda elevada incrementa riesgo |
| MonthlyIncome | -1 | Mayor ingreso ofrece resiliencia |
| NumberOfTimes90DaysLate | +1 | Atrasos graves elevan riesgo |
| DelinquencySum | +1 | Nuevas variables de severidad acumulada |
| SevereDelinquencyFlag | +1 | Indicador binario de crisis recurrente |
| IncomePerDependent | -1 | Más ingreso per cápita reduce estrés |
| CreditExposure | +1 | Exposición monetaria absoluta |
| UtilizationDelinquencyMix | +1 | Interacción riesgo: utilización + morosidad |

Las demás variables quedan con restricción neutra (`0`).

### Pipelines de Preprocesamiento
`SimpleImputer(median) → SMOTE → StandardScaler → Modelo`
- El mismo pipeline se comparte entre XGBoost y LightGBM para garantizar comparabilidad.
- SMOTE se aplica dentro del pipeline para evitar fugas de información.

In [11]:
X = df.drop(columns=[TARGET])
y = df[TARGET].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=RANDOM_STATE
)

def _normalize_feature_name(name: str) -> str:
    return ''.join(ch for ch in name.lower() if ch.isalnum())

monotonic_rules = {
    'revolvingutilizationofunsecuredlines': 1,
    'age': -1,
    'numberoftime3059dayspastduenotworse': 1,
    'debtratio': 1,
    'monthlyincome': -1,
    'numberofopencreditlinesandloans': 0,
    'numberoftimes90dayslate': 1,
    'numberrealestateloansorlines': 0,
    'numberoftime6089dayspastduenotworse': 1,
    'numberofdependents': 0,
    'delinquencysum': 1,
    'severedelinquencyflag': 1,
    'incomeperdependent': -1,
    'creditexposure': 1,
    'utilizationdelinquencymix': 1
}

monotonic_constraints = [
    monotonic_rules.get(_normalize_feature_name(col), 0)
    for col in X_train.columns
]

def make_pipeline(estimator):
    return ImbPipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('smote', SMOTE(random_state=RANDOM_STATE)),
        ('scaler', StandardScaler()),
        ('clf', estimator)
    ])

baseline_xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    tree_method='hist',
    learning_rate=0.05,
    max_depth=4,
    subsample=0.85,
    colsample_bytree=0.75,
    n_estimators=320,
    min_child_weight=4,
    gamma=0.5,
    reg_lambda=1.0,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    scale_pos_weight=3.0
)

monotonic_lgbm = LGBMClassifier(
    objective='binary',
    learning_rate=0.05,
    n_estimators=420,
    num_leaves=64,
    max_depth=-1,
    min_child_samples=45,
    subsample=0.85,
    colsample_bytree=0.8,
    monotone_constraints=monotonic_constraints,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

baseline_pipeline = make_pipeline(baseline_xgb)
monotonic_pipeline = make_pipeline(monotonic_lgbm)

print('Pipelines listos')

Pipelines listos


## Sección 3 — Entrenamiento Individual

- **BaselineXGB**: captura interacciones complejas sin restricciones. Objetivo principal: recall alto en morosos.
- **MonotonicLGBM**: mantiene coherencia causal y facilita auditorías. Esperamos AUC ligeramente menor pero decisiones más estables.
- Ambos modelos se entrenan sobre el dataset enriquecido con las cinco variables derivadas.
- Esta sección establece la línea base antes de introducir el gateway híbrido.

In [ ]:
baseline_pipeline.fit(X_train, y_train)
monotonic_pipeline.fit(X_train, y_train)

for name, model in {'BaselineXGB': baseline_pipeline, 'MonotonicLGBM': monotonic_pipeline}.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    print(f'{name} → Test AUC: {auc:.4f} | AUC-PR: {ap:.4f}')

## Sección 4 — Motor de Reglas y Gateway Híbrido

### RuleEngine
- Devuelve `1` (moroso evidente) cuando se cumplen señales fuertes: `DelinquencySum ≥ 4`, morosidad severa + alta exposición, o utilización >0.95 acompañada de atrasos.
- Devuelve `0` (pagador evidente) cuando el historial es limpio, la utilización es moderada y el ingreso per cápita supera la mediana.
- El resto de los casos quedan en `-1` (zona gris) para evaluación con modelos.

### HybridDecisionSystem
1. **Reglas primero**: si el RuleEngine clasifica 0 o 1, la decisión final se refuerza con el baseline clippeando la probabilidad (0.001-0.3 para buenos, 0.7-0.999 para malos).
2. **Alta confianza del baseline**: en zona gris, si XGBoost predice ≥0.85 o ≤0.15, se acepta su criterio directamente.
3. **MonotonicLGBM**: maneja los casos ambiguos restantes, beneficiándose de las restricciones monotónicas.

Este flujo mantiene interpretabilidad, aprovecha la potencia del baseline y deja la última palabra al modelo monotónico sólo cuando es necesario.

In [ ]:
class RuleEngine:
    def __init__(self, feature_names, income_median):
        self.feature_names = feature_names
        self.income_median = income_median

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            df_local = X[self.feature_names].copy()
        else:
            df_local = pd.DataFrame(X, columns=self.feature_names)

        labels = np.full(len(df_local), -1, dtype=int)

        high_risk = (
            (df_local['DelinquencySum'] >= 4)
            | ((df_local['SevereDelinquencyFlag'] == 1) & (df_local['CreditExposure'] > 3000))
            | ((df_local['RevolvingUtilizationOfUnsecuredLines'] > 0.95) & (df_local['DelinquencySum'] >= 2))
        )
        labels[high_risk] = 1

        low_risk = (
            (df_local['DelinquencySum'] == 0)
            & (df_local['SevereDelinquencyFlag'] == 0)
            & (df_local['RevolvingUtilizationOfUnsecuredLines'] < 0.4)
            & (df_local['DebtRatio'] < 0.45)
            & (df_local['IncomePerDependent'] > self.income_median)
        )
        labels[(low_risk) & (~high_risk)] = 0

        return labels

class HybridDecisionSystem(BaseEstimator, ClassifierMixin):
    def __init__(self, baseline_model, monotonic_model, rule_engine, baseline_high=0.85, baseline_low=0.15, threshold=0.5):
        self.baseline_model = baseline_model
        self.monotonic_model = monotonic_model
        self.rule_engine = rule_engine
        self.baseline_high = baseline_high
        self.baseline_low = baseline_low
        self.threshold = threshold

    def fit(self, X, y):
        # Los modelos ya están entrenados, solo devolvemos self para compatibilidad
        return self

    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X_df = X
        else:
            X_df = pd.DataFrame(X, columns=self.monotonic_model.named_steps['clf'].feature_name_)

        rule_labels = self.rule_engine.predict(X_df)
        proba_baseline = self.baseline_model.predict_proba(X_df)[:, 1]
        proba_monotonic = self.monotonic_model.predict_proba(X_df)[:, 1]

        final_proba = proba_monotonic.copy()
        mask_rule_default = rule_labels == 1
        mask_rule_good = rule_labels == 0
        final_proba[mask_rule_default] = np.clip(proba_baseline[mask_rule_default], 0.7, 0.999)
        final_proba[mask_rule_good] = np.clip(proba_baseline[mask_rule_good], 0.001, 0.3)

        ambiguous = rule_labels == -1
        high_conf_pos = (proba_baseline >= self.baseline_high) & ambiguous
        high_conf_neg = (proba_baseline <= self.baseline_low) & ambiguous
        final_proba[high_conf_pos] = proba_baseline[high_conf_pos]
        final_proba[high_conf_neg] = proba_baseline[high_conf_neg]

        return np.vstack([1 - final_proba, final_proba]).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= self.threshold).astype(int)

rule_engine = RuleEngine(feature_names=X_train.columns, income_median=X_train['IncomePerDependent'].median())
hybrid_gateway = HybridDecisionSystem(
    baseline_model=baseline_pipeline,
    monotonic_model=monotonic_pipeline,
    rule_engine=rule_engine,
    baseline_high=0.85,
    baseline_low=0.15,
    threshold=0.5
)

## Sección 5 — Evaluación Comparativa y Cobertura de Reglas

### Métricas Objetivo
- AUC-ROC ≥ 0.870 y AUC-PR ≥ 0.45 sobre el hold-out.
- Recall de morosos ≥ 0.70 para contener pérdidas de crédito.
- Cobertura de reglas entre 15% y 25% de los casos.

### Flujo de Evaluación
1. Medimos la cobertura del RuleEngine en el conjunto de test.
2. Calculamos AUC, AUC-PR, recall, precision y F1 para cada modelo.
3. Inspeccionamos la matriz de confusión y el `classification_report` para cada arquitectura.

El objetivo es que el gateway híbrido supere a los modelos individuales sin sacrificar claridad operativa.

In [ ]:
models_to_eval = {
    'BaselineXGB': baseline_pipeline,
    'MonotonicLGBM': monotonic_pipeline,
    'HybridGateway': hybrid_gateway
}

evaluation_rows = []

rule_assignments = rule_engine.predict(X_test)
rule_summary = pd.Series(rule_assignments).map({-1: 'Zona gris', 0: 'Pagador evidente', 1: 'Moroso evidente'}).value_counts()
print('Cobertura de reglas en test:')
display(rule_summary.to_frame('Observaciones'))

for name, model in models_to_eval.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    evaluation_rows.append({
        'Modelo': name,
        'AUC_ROC': auc,
        'AUC_PR': ap,
        'Recall': tp / (tp + fn + 1e-6),
        'Precision': tp / (tp + fp + 1e-6),
        'F1': (2 * tp) / (2 * tp + fp + fn + 1e-6),
        'TN': tn,
        'FP': fp,
        'FN': fn,
        'TP': tp
    })

    print(f"\n{name}")
    print(classification_report(y_test, y_pred, zero_division=0))

results_df = pd.DataFrame(evaluation_rows).sort_values(by='AUC_ROC', ascending=False)
display(results_df)


Cobertura de reglas en test:


,Observaciones
Zona gris,43061
Moroso evidente,1939


NameError: name 'f' is not defined

## Sección 6 — Persistencia y Despliegue

### Artefactos Guardados
Se serializa `models/hybrid_gateway_v1.pkl` con:
1. `baseline_pipeline`
2. `monotonic_pipeline`
3. `rule_engine`
4. `metadata` (umbrales, columnas esperadas, lista de features)

### Buenas Prácticas de Producción
- Replicar el feature engineering antes de llamar al gateway.
- Monitorear cobertura de reglas, distribución de probabilidades y métricas de rendimiento por capa.
- Versionar reglas y modelos por separado para mantener trazabilidad.

Ejecutar la celda final para almacenar el artefacto actualizado.

In [ ]:
artifact_path = MODELS_DIR / 'hybrid_gateway_v1.pkl'
joblib.dump({
    'baseline_pipeline': baseline_pipeline,
    'monotonic_pipeline': monotonic_pipeline,
    'rule_engine': rule_engine,
    'metadata': {
        'baseline_high': hybrid_gateway.baseline_high,
        'baseline_low': hybrid_gateway.baseline_low,
        'threshold': hybrid_gateway.threshold,
        'feature_columns': list(X_train.columns)
    }
}, artifact_path)
print(f'Gateway híbrido guardado en {artifact_path}')